In [ ]:
import pandas as pd
import re
try:
  import emoji
except:
  !pip install emoji
  import emoji

ModuleNotFoundError: ignored

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
ukraine_data = pd.read_csv('sample.csv')

In [ ]:
# funcs
def get_source(x_row):
    if x_row.src_idx != -1:
        src_range = (x_row.src_idx, x_row.transl_idx) if x_row.src_idx < x_row.transl_idx else (x_row.src_idx, len(x_row.message))
        return x_row.message[src_range[0]:src_range[1]].replace('source :', '').strip()
    else:
        return ''

def get_msg_text(x_row):
    if x_row.transl_idx != -1:
        src_range = (x_row.transl_idx, x_row.src_idx) if x_row.src_idx > x_row.transl_idx else (x_row.transl_idx, len(x_row.message))
        return x_row.message[src_range[0]:src_range[1]].replace('translation :', '').strip()
    else:
        return ''

def drop_og(msg_text):
    if 'original :' in msg_text:
        return msg_text[0:msg_text.index('original :')]
    else:
        return msg_text

def remove_links(msg_text):
    return re.sub(r'http[s]?://\S+', '', msg_text)

def remove_sm_tags(msg_text):
    #return re.sub(r'\s*(@\w+|#\w+)\s*', '', msg_text)
    return re.sub(r'\s*[@#]\w+\b', '', msg_text)

def clean_emojis(x_row):
    if len(x_row.emoji_expd) != 0:
        for ed in x_row.emoji_expd:
            x_row.msg_text = x_row.msg_text.replace(ed,'')
    return x_row.msg_text

In [ ]:
ukr_df = ukraine_data.copy()

In [ ]:
# pre-processing
ukr_df.message = ukr_df.message.apply(lambda x: x.replace('\n', ' ').lower())

In [ ]:
# generating indexes
ukr_df['src_idx'] = ukr_df.message.apply(lambda x: x.index('source :') if 'source :' in x else -1)
ukr_df['transl_idx'] = ukr_df.message.apply(lambda x: x.index('translation :') if 'translation :' in x else -1)

In [ ]:
# msg text extraction
ukr_df['msg_text'] = ukr_df.apply(get_msg_text, axis=1)
#msg text cleaning
ukr_df['msg_text'] = ukr_df.msg_text.apply(drop_og)
ukr_df['msg_text'] = ukr_df.msg_text.apply(remove_links)
ukr_df['msg_text'] = ukr_df.msg_text.apply(remove_sm_tags)
#source extraction
ukr_df['source'] = ukr_df.apply(get_source, axis=1)


In [ ]:
#emojis
ukr_df['emoji_data'] = ukr_df.msg_text.apply(lambda x: list(emoji.analyze(x)))
ukr_df['emoji_expd'] = ukr_df.emoji_data.apply(lambda x: [e_data.value.data['en'] for e_data in x])
ukr_df.msg_text = ukr_df.msg_text.apply(emoji.demojize)

In [ ]:
#text w/o emojis
ukr_df['noemoji_msg_text'] = ukr_df.apply(clean_emojis, axis=1)

In [ ]:
display(ukr_df.head(50))

,Unnamed: 0,message,annotation,src_idx,transl_idx,msg_text,source,emoji_data,emoji_expd,noemoji_msg_text
0,269608,"translation : the turkish government has once again urged greece to withdraw its military forces from the aegean islands, according to the associated press. ""there is an agreement, which greece violates. if greece does not stop this violation, the sovereignty of the islands will be discussed,"" said the head of the ministry of foreign affairs of turkey mevlut cavusoglu. subscribe to zn.ua source : https://t.me/znua_live/40978 zn.ua (зеркало недели. украина)",NaN,394,1,"the turkish government has once again urged greece to withdraw its military forces from the aegean islands, according to the associated press. ""there is an agreement, which greece violates. if greece does not stop this violation, the sovereignty of the islands will be discussed,"" said the head of the ministry of foreign affairs of turkey mevlut cavusoglu. subscribe to zn.ua",https://t.me/znua_live/40978 zn.ua (зеркало недели. украина),[],[],"the turkish government has once again urged greece to withdraw its military forces from the aegean islands, according to the associated press. ""there is an agreement, which greece violates. if greece does not stop this violation, the sovereignty of the islands will be discussed,"" said the head of the ministry of foreign affairs of turkey mevlut cavusoglu. subscribe to zn.ua"
1,181446,"source : pravda gerashchenko translation : ""their own religion was created for azov fighters"": putin's propaganda has hit another bottom details: the bunker führer's bunker cisterns keep broadcasting nonsense from the zombie tvs. ""eating babies"" is no longer relevant. the bloodthirsty ""heathens"" are coming into play! #♪ over the fence ♪ telegram | telegram en | youtube | send photo/video https://t.me/pravda_gerashchenko/13205",NaN,1,30,"""their own religion was created for azov fighters"": putin's propaganda has hit another bottom details: the bunker führer's bunker cisterns keep broadcasting nonsense from the zombie tvs. ""eating babies"" is no longer relevant. the bloodthirsty ""heathens"" are coming into play! #♪ over the fence ♪ telegram | telegram en | youtube | send photo/video",pravda gerashchenko,[],[],"""their own religion was created for azov fighters"": putin's propaganda has hit another bottom details: the bunker führer's bunker cisterns keep broadcasting nonsense from the zombie tvs. ""eating babies"" is no longer relevant. the bloodthirsty ""heathens"" are coming into play! #♪ over the fence ♪ telegram | telegram en | youtube | send photo/video"
2,119486,"translation : boris johnsonuk, a glorious cossack and friend of ukraine, is again in ukraine. he wrote a joyful letter saying that he is glad to be back in kyiv. within a couple of years, the tweet had already gathered tens of thousands of supporters. after this tweet, boris johnson also posted a video. of course, this kozak never arrives without gifts for ukraine, so he is welcomed and thanked by the people of great britain. @informnapalm source : https://t.me/informnapalm/9560 informnapalm",NaN,447,1,"boris johnsonuk, a glorious cossack and friend of ukraine, is again in ukraine. he wrote a joyful letter saying that he is glad to be back in kyiv. within a couple of years, the tweet had already gathered tens of thousands of supporters. after this tweet, boris johnson also posted a video. of course, this kozak never arrives without gifts for ukraine, so he is welcomed and thanked by the people of great britain.",https://t.me/informnapalm/9560 informnapalm,[],[],"boris johnsonuk, a glorious cossack and friend of ukraine, is again in ukraine. he wrote a joyful letter saying that he is glad to be back in kyiv. within a couple of years, the tweet had already gathered tens of thousands of supporters. after this tweet, boris johnson also posted a video. of course, this kozak never arrives without gifts for ukraine, so he is welcomed and thanked by the people of 

In [ ]:
#post-processing
ukr_df.drop(['src_idx', 'transl_idx'], axis=1,inplace=True)

In [ ]:
ukr_df.rename(columns={'Unnamed: 0':'corp_index'})

,corp_index,message,annotation,msg_text,source
0,269608,translation : the turkish government has once...,NaN,the turkish government has once again urged gr...,https://t.me/znua_live/40978 zn.ua (зеркало не...
1,181446,"source : pravda gerashchenko translation : ""t...",NaN,"""their own religion was created for azov fight...",pravda gerashchenko
2,119486,"translation : boris johnsonuk, a glorious cos...",NaN,"boris johnsonuk, a glorious cossack and friend...",https://t.me/informnapalm/9560 informnapalm
3,179702,"translation : near kherson, the afu destroyed...",NaN,"near kherson, the afu destroyed 70 russian sol...",https://t.me/antiitforce/14974 atf•anti terror...
4,248673,source : инсайдер ua translation : ⚡️zelensky...,NaN,⚡️zelensky announced preparations for the seco...,инсайдер ua
...,...,...,...,...,...
195,72050,translation : azov man denis prokopenka's gir...,NaN,azov man denis prokopenka's girlfriend told us...,https://t.me/ukrainenow/16511 ukraine now
196,128735,translation : shakhtar footballer knocked out...,NaN,shakhtar footballer knocked out a drunken prov...,https://t.me/u_now/85354 украина сейчас: новос...
197,315024,translation : our military are currently trai...,NaN,our military are currently training in the reg...,https://t.me/kievreal1/36085 реальный киев | н...
198,172226,translation : source : https://t.me/novostin...,NaN,,https://t.me/novostiniko/24722 новости николае...


In [ ]:
ukr_df.to_csv('sample_clean.csv')